# TIBCO BE PoC
This demonstrates how to use Assistant API to generate BE rules.

## Setup
First install openai if it has not been done already, i.e.,
```
!pip install openai
```

Second, create an API key for the project in the OpenAI playground, and then define env variable and restart the notebook, i.e.,
```
export OPENAI_API_KEY="sk-proj-..."
jupyter notebook
```

Load the API key and relevant Python libaries.

In [12]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

## Upload BE Documentation of Standard Functions
OpenAI LLM can search files in a vector store for documentations required when processing user messages.  We use this mechanism of [embeddings](https://platform.openai.com/docs/guides/embeddings) to provide documentations of BE catalog functions.

First, construct a JSON file that contains descriptions of all BE catalog functions used by the AI assistant, e.g., 
```
[
    {
        "Function": "Log.getLogger()",
        "Signature": "Object getLogger (String loggerName)",
        "Domain": "ACTION, CONDITION, QUERY, BUI",
        "Description": "Get a logger to be used inside a rule/rulefunction name.",
        "Parameters": [
            {
                "name": "loggerName",
                "type": "String",
                "description": "Name of the Logger to get. Use the project path of the current rule or rulefunction."
            }
        ],
        "Returns": {
            "Type": "Object"
        },
        "Cautions": "none",
        "Example": "Object logger = Log.getLogger(\"Rules.VerifyCreditRF\");"
    },
]
```
We'll upload this file to a vector store next.

Check if the target vector store exists already:

In [ ]:
# fetch current vector stores
curr_stores = client.beta.vector_stores.list()
vector_store = None
for x in curr_stores.data:
  print(x.name, x.file_counts)
  if x.name == "BE Assistant Store":
    vector_store = x
if vector_store is not None:
  print(vector_store.name)

If the store does not exist, we create a vector store, and upload the documentation file into the store.

In [ ]:
if vector_store is None:
  # Create a vector store for BE docs
  vector_store = client.beta.vector_stores.create(name="BE Assistant Store")

  # Ready the files for upload to OpenAI
  file_paths = ["Standard.json", "CEP.json", "RDBMS.json"]
  file_streams = [open(path, "rb") for path in file_paths]
 
  # Use the upload and poll SDK helper to upload the files, add them to the vector store,
  # and poll the status of the file batch for completion.
  file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
  )
 
  # You can print the status and the file counts of the batch to see the result of this operation.
  print(file_batch.status)
  print(file_batch.file_counts)

## Create an AI Assistant
We can then build an AI assistant for writing BE rules and functions.

We first specify an overview for the syntax of BE rules and functions, and a few code examples, and then use the instruction to create an assistant, and attach the previously created vector store containing BE documentation files.

In [21]:
# Instructions for BE Assistant
code_instruction="""
You are an assistant.  You write TIBCO BE rules and functions according to user requests.

A valid TIBCO BE function must not violate any of the following requirements:

A function must include a curly braces block of `attribute` that specifies the setting of `validity`.
A function must include a curly braces block of `scope` that lists all input variables that the function may read and update.
A function must include a block of `body` that is the code for activities required by user.

Functions may include calls to BE library functions.  You may lookup definitions of BE library functions via fine-tuning or embedding docs.

Example of a BE function for pre-processing an event of type SubmitOrder:
```
/**
 * @description 
 * @author AI Assistant
 */
void rulefunction RuleFunctions.onSubmitOrder {
	attribute {
		validity = ACTION;
	}
	scope {
		Events.SubmitOrder evt;
	}
	body {
		Object logger = Log.getLogger("RuleFunctions.onSubmitOrder");
		
		Concepts.ShoppingCart cart = Instance.getByExtIdByUri(RuleFunctions.extIdPrefix("ShoppingCart")+evt.user, "/Concepts/ShoppingCart");
		if (cart == null) {
			Log.log(logger, "info", "Shopping cart of %s does not exist", evt.user);
			// reply 404 Shopping cart not found
			Event.replyEvent(evt, Events.HTTPStatus.HTTPStatus(null, null, "404"));
			Event.consumeEvent(evt);
			return;
		}
		
		if (cart.items@length == 0) {
			Log.log(logger, "info", "Shopping cart of %s is empty", evt.user);
			// reply 400 Shopping cart is empty
			Event.replyEvent(evt, Events.HTTPStatus.HTTPStatus(null, null, "400"));
			Event.consumeEvent(evt);
			return;
		}
		Log.log(logger, "info", "Creating new order for user %s", evt.user);
	}
}
```

To load a concept from cache, you can call BE library functions in the package of `Cluster.DataGrid`, e.g.,
```
    Concepts.ShoppingCart cart = Cluster.DataGrid.CacheLoadConceptById(oid, true);
```

To get a cluster-wide lock, you can also call BE libary function in the package of `Cluster.DataGrid`, e.g.,
```
    boolean lockAcquired = Cluster.DataGrid.Lock(key, 500, false);
```

The following example constructs a JSON text by using BE library functions for strings:
```
/**
 * @description
 * @author AI Assistant
 */
String rulefunction RuleFunctions.userToJSON {
	attribute {
		validity = ACTION;
	}
	scope {
		Concepts.User user;
	}
	body {
		Object buff = String.createBuffer(300);
		String.append(buff, String.format("{\"name\":\"%s\",", user.name));
		String.append(buff, String.format("\"status\":\"%s\",", user.status));
		String.append(buff, String.format("\"rewards\":%d,", user.rewardPoints));
		String.append(buff, String.format("\"orders\":%d,", user.orders@length));
		String.append(buff, "\"coupons\":[");
		for (int i = 0; i < user.coupons@length; i++) {
			String.append(buff, String.format("\"%s\"", RuleFunctions.couponToString(user.coupons[i])));
			if (i != user.coupons@length-1) {
				String.append(buff, ",");
			}
		}
		String.append(buff, "]}");
		return String.convertBufferToString(buff);
	}
}
```

A valid TIBCO BE rule must not violate any of the following requirements:

A rule must include a curly braces block of `attribute` that specifies settings of `priority` and `forwardChain`.
A rule must include a curly braces block of `declare` that lists all input variables used by the rule conditions.
A rule must include a block of `when` that lists rule condisions for matching properties of the declared variables.  Variables must not be declared in this block.
A rule must include a block of `then` that is the code for required actions of the rule.

Rules may create new instances of a Concept class by using a constructor function that is named by its class name.  The arguments of a concept constructor includes an unique extId and values of all properties specified by the Concept class definition, e.g.,
```
Object item = Concepts.ShoppingItem.ShoppingItem(
	null /*extId String */,
	evt.sku /*sku String */,
	evt.unit /*unit long */,
	amount /*amount double */);
```

Rules may create new instances of an Event class by using a constructor function that is named by its class name.  The arguments of an event constructor includes an unique extId, a event payload string, and values of all properties specified by the Event class definition, e.g.,
```
Object evt = Events.HTTPStatus.HTTPStatus(
    null /*extId String*/,
    RuleFunctions.cartToJSON(cart) /*payload String */,
    "200" /*status String */);
```

Rules may include calls to BE library functions.

Example of a rule that acts on an event of type AddItemToCart, and adds a matching product to a matching shopping cart:
```
/**
 * @description 
 * @author AI Assistant
 */
rule Rules.AddItemToCart {
	attribute {
		priority = 5;
		forwardChain = true;
	}
	declare {
		Events.AddItemToCart evt;
		Concepts.Product prod;
		Concepts.ShoppingCart cart;
	}
	when {
		prod.sku == evt.sku;
		cart.userName == evt.user;
	}
	then {
		Object logger = Log.getLogger("Rules.AddItemToCart");
		double amount = evt.unit * prod.price;
		cart.items[cart.items@length] = Concepts.ShoppingItem.ShoppingItem(
			null /*extId String */,
			evt.sku /*sku String */,
			evt.unit /*unit long */,
			amount /*amount double */);
		cart.amount += amount;
		Log.log(logger, "info", "Added %d units of SKU %s to shopping cart of %s", evt.unit, evt.sku, evt.user);
		Event.replyEvent(evt, Events.HTTPStatus.HTTPStatus(null,RuleFunctions.cartToJSON(cart),"200"));
		Event.consumeEvent(evt);
	}
}
```

Keep your response brief with only the generated code for a BE rule or function.
"""

In [ ]:
# convert code instruction into a string to be used in Continue vsCode configuration
import json

data = {"systemMessage": code_instruction}
json_string = json.dumps(data)
print(json_string)

In [19]:
def createAssistant(instruction, store, model="gpt-4o", name="BE Assistant"):
  # fetch existing assistants
  assistants = client.beta.assistants.list(
    order="desc",
    limit="10",
  )

  # return assistant matching specified name if it exists already
  matched = [x for x in assistants.data if x.name == name]
  if len(matched) > 0:
    print(f"{name} already exists: {matched[0].id}")
    return matched[0]

  # create a new assistant
  assistant = None
  if store is not None:
    assistant = client.beta.assistants.create(
      name=name,
      instructions=instruction,
      model=model,
      temperature=0.01,
      tools=[{"type": "file_search"}],
      tool_resources={
        "file_search": {
          "vector_store_ids": [store]
        }
      }
    )
  else:
    assistant = client.beta.assistants.create(
      name=name,
      instructions=instruction,
      model=model,
      temperature=0.01
    )

  print(f"{name} is created: {assistant.id}")
  return assistant

In [ ]:
# create assistant
assistant = createAssistant(code_instruction, vector_store.id)
print(assistant.name, assistant.model)

### Define Async Event Handler
We use asynchronous streaming to process AI responses here.  Synchronous runner APIs also exist for users to chat with the assistant.

In [23]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# Create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'file_search':
      if delta.file_search.input:
        print(delta.file_search.input, end="", flush=True)
      if delta.file_search.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.file_search.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

## Create Thread and Run for a Request
User can create a chat thread, send a request to LM, and use the above event handler to printout the response from LM.

In the following function, we always create a new thread for each request.  You may reuse the same thread to send multiple requests to the assistant.  However, all previous messages will also be sent again if the same thread is used, and thus it would save costs if a new thread is created for each request except for cases where a follow-up request is related to the previous one.

In [24]:
def generateCode(request, name="Tuned BE Assistant"):
  # fetch existing BE assistant id
  assistants = client.beta.assistants.list(
    order="desc",
    limit="10",
  )
  beAssistants = [x for x in assistants.data if x.name == name]
  if len(beAssistants) <= 0:
    print(f"{name} does not exist")
    return
  assistantId = beAssistants[0].id
  print("Assistant:", assistantId)

  # create thread and message
  thread = client.beta.threads.create()
  message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=request
  )

  # create run stream, and printout response
  with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistantId,
    event_handler=EventHandler(),
  ) as stream:
    stream.until_done()

In [25]:
# Request for a typical BE rule
request = """
Write a rule RestockInventory to handle an event of type Events.RestockInventory.

The rule executes when the sku property of the event matches the sku of a concept of type Concepts.Inventory. It will perform the following actions:

1. Set the inStock count of the concept to the amount specified by the event;
2. Write a log message containing the sku and restock amount;
3. Create a response event using the event constructor of HTTPStatus with a payload denoting successful completion and an HTTP status of "200";
4. Send the reply event;
5. Consume the event.
"""
generateCode(request)

Assistant: asst_Qg8AWyt8qmBDr0su2cgJwjmS

assistant > ```java
/**
 * @description Restock inventory based on the event details and send a response.
 */
rule Rules.RestockInventory {
	attribute {
		priority = 5;
		forwardChain = true;
	}
	declare {
		Events.RestockInventory evt;
		Concepts.Inventory inventory;
	}
	when {
		inventory.sku == evt.sku;
	}
	then {
		Object logger = Log.getLogger("Rules.RestockInventory");
		inventory.inStock = evt.amount;
		Log.log(logger, "info", "Restocked SKU %s with amount %d", evt.sku, evt.amount);
		Object responseEvent = Events.HTTPStatus.HTTPStatus(
			null /*extId String*/,
			String.format("Restock completed for SKU %s with amount %d", evt.sku, evt.amount) /*payload String*/,
			"200" /*status String*/);
		Event.replyEvent(evt, responseEvent);
		Event.consumeEvent(evt);
	}
}
```

## More Requests for BE Rules
Following example creates a BE rule that uses library functions for files.

In [26]:
# A sample BE rule that writes a file
request = """
Write a rule Rules.TestFile that executes on any event of type WriteConcept. The rule will perform the following actions:

1. Acquire a cluster-wide lock on the conceptId specified by the event, with a timeout of 500 ms;
2. If failed to acquire the lock, log the error of failed lock; otherwise,
3. Load a concept of type Concepts.Inventory from BE cache by using the specified conceptId;
4. Lookup a file path specified by a system global variable of name "LogFile";
5. Write a log message containing the conceptId and the file path;
6. Open the file for read and write by using the path;
7. Serialize the loaded concept into JSON string, and then write the result to the file;
8. Close the file;
9. consume the event.
"""
generateCode(request)

Assistant: asst_Qg8AWyt8qmBDr0su2cgJwjmS

assistant > ```java
/**
 * @description 
 * @author AI Assistant
 */
rule Rules.TestFile {
	attribute {
		priority = 5;
		forwardChain = true;
	}
	declare {
		Events.WriteConcept evt;
	}
	when {
	}
	then {
		Object logger = Log.getLogger("Rules.TestFile");
		String conceptId = evt.conceptId;
		boolean lockAcquired = Cluster.DataGrid.Lock(conceptId, 500, false);
		
		if (!lockAcquired) {
			Log.log(logger, "error", "Failed to acquire lock for conceptId: %s", conceptId);
		} else {
			Concepts.Inventory inventory = Cluster.DataGrid.CacheLoadConceptById(conceptId, true);
			String filePath = System.getGlobalVariableAsString("LogFile");
			Log.log(logger, "info", "ConceptId: %s, FilePath: %s", conceptId, filePath);
			
			Object file = File.open(filePath, "rw");
			String json = JSON.serializeConcept(inventory);
			File.write(file, json);
			File.close(file);
		}
		
		Event.consumeEvent(evt);
	}
}
```

In [31]:
# Use assistent with embeddings to create a sample BE rule that writes a file
request = """
Write a rule Rules.TestFile that executes on any event of type WriteConcept. The rule will perform the following actions:

1. Acquire a cluster-wide lock on the conceptId specified by the event, with a timeout of 500 ms;
2. If failed to acquire the lock, log the error of failed lock; otherwise,
3. Load a concept of type Concepts.Inventory from BE cache by using the specified conceptId;
4. Lookup a file path specified by a system global variable of name "LogFile";
5. Write a log message containing the conceptId and the file path;
6. Open the file for read and write by using the path;
7. Serialize the loaded concept into JSON string, and then write the result to the file;
8. Close the file;
9. consume the event.
"""
generateCode(request, "BE Assistant")

Assistant: asst_HTJotqGH9r253VbuzB1yyJKd

assistant > file_search


assistant > ```java
/**
 * @description Rule to handle WriteConcept events and perform specified actions
 * @ruleauthor AI Assistant
 */
rule Rules.TestFile {
	attribute {
		priority = 5;
		forwardChain = true;
	}
	declare {
		Events.WriteConcept evt;
		Concepts.Inventory inventory;
	}
	when {
		// No specific conditions, rule triggers on any WriteConcept event
	}
	then {
		Object logger = Log.getLogger("Rules.TestFile");
		String conceptId = evt.conceptId;
		boolean lockAcquired = Cluster.DataGrid.Lock(conceptId, 500, false);
		
		if (!lockAcquired) {
			Log.log(logger, "error", "Failed to acquire lock for conceptId: %s", conceptId);
		} else {
			inventory = Cluster.DataGrid.CacheLoadConceptById(conceptId, true);
			String logFilePath = System.getGlobalVariableAsString("LogFile", "/default/path/to/logfile");
			Log.log(logger, "info", "Processing conceptId: %s, Log file path: %s", conceptId, logFilePath);
			
			Syst

## Request for BE Functions
User may also send requests to write BE functions.  Following is a simple function.

In [27]:
# Simple function
request = """
write a function to load Order concept by specified Id from cache.  create new if it does not exist.
"""
generateCode(request)

Assistant: asst_Qg8AWyt8qmBDr0su2cgJwjmS

assistant > ```java
/**
 * @description Load Order concept by specified Id from cache. Create new if it does not exist.
 * @author AI Assistant
 */
Concepts.Order rulefunction RuleFunctions.loadOrCreateOrder {
	attribute {
		validity = ACTION;
	}
	scope {
		String oid;
	}
	body {
		Concepts.Order order = Cluster.DataGrid.CacheLoadConceptById(oid, true);
		if (order == null) {
			order = Concepts.Order.Order(oid /*extId*/, /* initialize other properties if needed */);
		}
		return order;
	}
}
```

Following is a typical BE processor function

In [28]:
# Preprocessor function
request = """
Write a function to preprocess an OrderStatus event.

Acquire a cluster-wide lock on the orderId specified by the event, then load an Order concept from cache by using the specified orderId.  

Log the status of the operations. Do not unlock the outstanding lock.
"""
generateCode(request)

Assistant: asst_Qg8AWyt8qmBDr0su2cgJwjmS

assistant > ```java
/**
 * @description Preprocess OrderStatus event by acquiring lock and loading Order concept
 * @author AI
 */
void rulefunction RuleFunctions.onOrderStatus {
	attribute {
		validity = ACTION;
	}
	scope {
		Events.OrderStatus evt;
	}
	body {
		Object logger = Log.getLogger("RuleFunctions.onOrderStatus");

		// Acquire cluster-wide lock
		boolean lockAcquired = Cluster.DataGrid.Lock(evt.orderId, 500, false);
		Log.log(logger, "info", "Lock acquired on orderId %s: %b", evt.orderId, lockAcquired);

		if (lockAcquired) {
			// Load Order concept from cache
			Concepts.Order order = Cluster.DataGrid.CacheLoadConceptById(evt.orderId, true);
			if (order != null) {
				Log.log(logger, "info", "Order loaded from cache: %s", evt.orderId);
			} else {
				Log.log(logger, "warn", "Order not found in cache: %s", evt.orderId);
			}
		} else {
			Log.log(logger, "warn", "Failed to acquire lock on orderId: %s", evt.orderId);
		}
	}
}
```

Following is a function using BE library functions for string manipulation.

In [29]:
# Function for string manipulation
request = """
Write a function `Functions.countWord` that demonstrates string functions. The function inputs a text content of a document and a word to be searched.  It performs the following tasks:

1. Find the first position of the word in the content of the document;
2. Log the first position of the word found;
3. Find the next occurrence of the word in the content, and increase the count of the occurrences;
4. Loop until no more occurrences of the word is found;
5. Log the total count of the word occurrences;
6. Return the total count.
"""
generateCode(request)

Assistant: asst_Qg8AWyt8qmBDr0su2cgJwjmS

assistant > ```java
/**
 * @description Count occurrences of a word in the document content and log details.
 * @author AI
 */
int rulefunction Functions.countWord {
	attribute {
		validity = ACTION;
	}
	scope {
		String content;
		String word;
	}
	body {
		Object logger = Log.getLogger("Functions.countWord");
		int count = 0;
		int position = String.indexOf(content, word);
		
		if (position == -1) {
			Log.log(logger, "info", "Word not found in the content.");
			return count;
		}
		
		Log.log(logger, "info", "First position of the word found at: %d", position);
		count++;
		
		while (position != -1) {
			position = String.indexOf(content, word, position + 1);
			if (position != -1) {
				count++;
			}
		}
		
		Log.log(logger, "info", "Total count of the word occurrences: %d", count);
		return count;
	}
}
```

In [32]:
# Use model with embedding to create function of string manipulation
request = """
Write a function `Functions.countWord` that demonstrates string functions. The function inputs a text content of a document and a word to be searched.  It performs the following tasks:

1. Find the first position of the word in the content of the document;
2. Log the first position of the word found;
3. Find the next occurrence of the word in the content, and increase the count of the occurrences;
4. Loop until no more occurrences of the word is found;
5. Log the total count of the word occurrences;
6. Return the total count.
"""
generateCode(request, "BE Assistant")

Assistant: asst_HTJotqGH9r253VbuzB1yyJKd

assistant > file_search


assistant > ```java
/**
 * @description Function to count occurrences of a word in a document content and log the details.
 * @param content The text content of the document.
 * @param word The word to be searched in the content.
 * @return The total count of the word occurrences.
 */
int rulefunction Functions.countWord {
    attribute {
        validity = ACTION;
    }
    scope {
        String content;
        String word;
    }
    body {
        Object logger = Log.getLogger("Functions.countWord");
        int count = 0;
        int position = String.indexOfString(content, 0, word);
        
        if (position == -1) {
            Log.log(logger, "info", "Word '%s' not found in the content.", word);
            return count;
        }
        
        Log.log(logger, "info", "First occurrence of word '%s' found at position %d.", word, position);
        count++;
        
        while (true) {
            posit

Following is a function using BE library functions for RDBMS query.

In [30]:
# Function for RDBMS query
request = """
Write a function `Functions.executeQuery` that demonstrates RDBMS functions. The function inputs a jdbcResourceURI, a SQL string, and a conceptURI for returing concepts.  It performs the following tasks:

1. Set the current RDBMS connection to the specified jdbcResourceURI;
2. Log the SQL statement to be executed;
3. Execute the query;
4. Log the number of concepts returned by the query;
5. Release the RDBMS connection;
6. Return the list of concepts from the RDBMS query.
"""
generateCode(request)

Assistant: asst_Qg8AWyt8qmBDr0su2cgJwjmS

assistant > ```java
/**
 * @description Demonstrates RDBMS functions by executing a SQL query and returning concepts.
 * @author AI
 */
Object[] rulefunction Functions.executeQuery {
	attribute {
		validity = ACTION;
	}
	scope {
		String jdbcResourceURI;
		String sql;
		String conceptURI;
	}
	body {
		Object logger = Log.getLogger("Functions.executeQuery");
		Log.log(logger, "info", "Setting RDBMS connection to: %s", jdbcResourceURI);
		RDBMS.setConnection(jdbcResourceURI);

		Log.log(logger, "info", "Executing SQL: %s", sql);
		Object[] results = RDBMS.executeQuery(sql, conceptURI);

		Log.log(logger, "info", "Number of concepts returned: %d", results@length);
		RDBMS.releaseConnection();

		return results;
	}
}
```

In [33]:
# Use model with embeddings to create function for RDBMS query
request = """
Write a function `Functions.executeQuery` that demonstrates RDBMS functions. The function inputs a jdbcResourceURI, a SQL string, and a conceptURI for returing concepts.  It performs the following tasks:

1. Set the current RDBMS connection to the specified jdbcResourceURI;
2. Log the SQL statement to be executed;
3. Execute the query;
4. Log the number of concepts returned by the query;
5. Release the RDBMS connection;
6. Return the list of concepts from the RDBMS query.
"""
generateCode(request, "BE Assistant")

Assistant: asst_HTJotqGH9r253VbuzB1yyJKd

assistant > file_search


assistant > ```java
/**
 * @description Executes a SQL query using the specified JDBC resource URI and returns the resulting concepts.
 * @param jdbcResourceURI The JDBC resource URI for the database connection.
 * @param sql The SQL query string to execute.
 * @param conceptURI The URI of the concept type to return.
 * @return A list of concepts resulting from the SQL query.
 */
Concept[] rulefunction Functions.executeQuery {
    attribute {
        validity = ACTION;
    }
    scope {
        String jdbcResourceURI;
        String sql;
        String conceptURI;
    }
    body {
        // Set the current RDBMS connection
        Database.setCurrentConnection(jdbcResourceURI);
        
        // Log the SQL statement to be executed
        Object logger = Log.getLogger("Functions.executeQuery");
        Log.log(logger, "info", "Executing SQL: %s", sql);
        
        // Execute the query
        Concept[] results

## AI Model Fine-tuning
When BE library functions use multi-level nested packages, or multiple functions exist for similar tasks such as load concepts from cache or from memory, LM may not be able to decide which function to call, and it may also make up a hallucinated Java function calls.  To correct such problems, we try to fine-tune the AI model as follows. 

### Validate Dataset for Fine-tuning
#### Load dataset for tuning

In [5]:
import json
data_path = "Standard.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 706
First example:
{'role': 'system', 'content': 'BE Assistant'}
{'role': 'user', 'content': 'This method returns the maximum history value since the specified time.'}
{'role': 'assistant', 'content': 'Object Temporal.Numeric.maxSince(PropertyAtom pa, long time)'}


#### Validate data format

In [6]:
from collections import defaultdict

# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
            print(messages)
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


#### Data warnings and token counts

In [ ]:
!pip install tiktoken

In [7]:
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [8]:
# Estimate size of training dataset
with open("Standard.jsonl", "r") as f:
    content = f.read()
    print(len(encoding.encode(content)))

50923


In [ ]:
!pip install numpy

In [9]:
import numpy as np

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 16385 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 22, 233
mean / median: 49.89376770538244, 43.0
p5 / p95: 31.0, 80.5

#### Distribution of num_assistant_tokens_per_example:
min / max: 4, 37
mean / median: 13.242209631728045, 12.0
p5 / p95: 7.0, 20.0

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning


#### Estimate cost

In [10]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 16385

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~35225 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~105675 tokens


### Upload Training File

In [13]:
trainingFile = client.files.create(
  file=open(data_path, "rb"),
  purpose="fine-tune"
)
print(trainingFile.id)

file-RrR3mRJU8DdU9Uq6iWji5ClP


### Start Training Job

In [15]:
tuningJob = client.fine_tuning.jobs.create(
  training_file=trainingFile.id, 
  model="gpt-4o-2024-08-06",
  suffix="tibco-be",
  hyperparameters={
    "n_epochs": 4
  }
)
print(tuningJob.id)

ftjob-czwfG3aWn5odlW4mshx6AQzv


### Check Training Status

In [18]:
# Retrieve the state of a fine-tune
state = client.fine_tuning.jobs.retrieve(tuningJob.id)
print(state.status, state.trained_tokens, state.fine_tuned_model)

succeeded 136476 ft:gpt-4o-2024-08-06:personal:tibco-be:A3t3UIIn


### Create Assistant with Fine-tuned Model

In [22]:
assistant = createAssistant(code_instruction, None, state.fine_tuned_model, "Tuned BE Assistant")
print(assistant.name, assistant.model)

Tuned BE Assistant is created: asst_Qg8AWyt8qmBDr0su2cgJwjmS
Tuned BE Assistant ft:gpt-4o-2024-08-06:personal:tibco-be:A3t3UIIn


## Create Assistant with Earlier Model

In [ ]:
assistant = createAssistant(code_instruction, vector_store.id, "gpt-3.5-turbo-0125", "BE Assistant 3.5")
print(assistant.name, assistant.model)